In [1]:
from bs4 import BeautifulSoup
# import pandas as pd
from datetime import datetime, timedelta
from itertools import combinations
import json
import requests
import math
from typing import Optional, List, Dict

from pydantic import BaseModel
from typing import Mapping
import numpy as np
import polars as pl
import hvplot.polars
import pandas as pd
import altair as alt
import math

from shared.utils import formalize_data_with_str
from shared.random_rules import RandomRules

In [2]:
rr = RandomRules(
  TOTAL_NUMBERS = 35,
  CHOOSE = 5,
  MAX_PRIMES = 3,
  N_VALUES=["1", "2", "3"],
  # Rangos de valores de las bolillas
  MIN_B_VALUES = [1, 2, 3, 4, 5],
  MAX_B_VALUES = [31, 32, 33, 34, 35]
)

In [3]:
url = 'https://resultados.latinka.com.pe/i.do?m=historico&t=0&s=164'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

tables = soup.select('table')
rows = tables[1].find_all('tr')

In [4]:
lst = []
post_str_b1 = None
post_str_b2 = None
post_str_b3 = None
post_str_b4 = None
post_str_blast = None
for row in rows:
  cols = [data.text.strip() for data in row.find_all('td')]
  
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=cols[2])
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_b2)
  cols.append(post_str_b3)
  cols.append(post_str_b4)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_b2 = str_combo[1]
  post_str_b3 = str_combo[2]
  post_str_b4 = str_combo[3]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

dfRows = pl.DataFrame(
  lst,
  schema=[
    'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_b2', 'post_str_b3', 'post_str_b4', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(dfRows)
print(f'{TOTAL_LINES=}')
display(dfRows.head(10))

TOTAL_LINES=7731


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""15/04/2025""","""4187""","""29 15 23 33 10""","""1015232933""","[10, 15, … 33]",10,15,23,29,33,"""10""","""15""","""23""","""29""","""33""",null,null,null,null,null,false,0,"""05_08_06_04""",5
"""14/04/2025""","""4186""","""09 27 16 14 04""","""0409141627""","[4, 9, … 27]",4,9,14,16,27,"""04""","""09""","""14""","""16""","""27""","""10""","""15""","""23""","""29""","""33""",true,24,"""05_05_02_11""",15
"""13/04/2025""","""4185""","""26 06 13 09 27""","""0609132627""","[6, 9, … 27]",6,9,13,26,27,"""06""","""09""","""13""","""26""","""27""","""04""","""09""","""14""","""16""","""27""",false,0,"""03_04_13_01""",-29
"""12/04/2025""","""4184""","""23 12 10 16 03""","""0310121623""","[3, 10, … 23]",3,10,12,16,23,"""03""","""10""","""12""","""16""","""23""","""06""","""09""","""13""","""26""","""27""",false,0,"""07_02_04_07""",-6
"""11/04/2025""","""4183""","""09 10 08 01 12""","""0108091012""","[1, 8, … 12]",1,8,9,10,12,"""01""","""08""","""09""","""10""","""12""","""03""","""10""","""12""","""16""","""23""",true,24,"""07_01_01_02""",-5
"""10/04/2025""","""4182""","""30 33 34 10 21""","""1021303334""","[10, 21, … 34]",10,21,30,33,34,"""10""","""21""","""30""","""33""","""34""","""01""","""08""","""09""","""10""","""12""",false,0,"""11_09_03_01""",8
"""09/04/2025""","""4181""","""29 21 20 28 04""","""0420212829""","[4, 20, … 29]",4,20,21,28,29,"""04""","""20""","""21""","""28""","""29""","""10""","""21""","""30""","""33""","""34""",true,16,"""16_01_07_01""",-33
"""08/04/2025""","""4180""","""10 20 30 05 16""","""0510162030""","[5, 10, … 30]",5,10,16,20,30,"""05""","""10""","""16""","""20""","""30""","""04""","""20""","""21""","""28""","""29""",false,0,"""05_06_04_10""",11
"""07/04/2025""","""4179""","""08 21 22 24 07""","""0708212224""","[7, 8, … 24]",7,8,21,22,24,"""07""","""08""","""21""","""22""","""24""","""05""","""10""","""16""","""20""","""30""",true,16,"""01_13_01_02""",37


In [5]:
dfRows.item(0, 'b2')

15

In [6]:
import holoviews as hv
def plot_post_col (col: str, post_col: str, test_value: str | None = None):
  if test_value == None:
    test_value = dfRows.item(0, col)
    print(f'Setting {test_value=}')

  df_viz = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      post_col
    ).agg(pl.count(post_col).alias('count')) \
    .sort("count", descending=True)
  
  
  df_10_last = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .head(10)
  
  # Agrega una columna que sea el porcentaje de la frecuencia relativa
  total = df_viz['count'].sum()
  df_viz = df_viz.with_columns(
    ( (pl.col('count') / total) * 100 ).alias('percent')
  )
  mean_percent = df_viz['percent'].mean()
  mean_percent_75 = mean_percent * 75 / 50
  mean_percent_25 = mean_percent * 25 / 50

  chart = df_viz.hvplot.bar(
    x=post_col,
    y='percent'
  )

  chart = chart * hv.HLine(mean_percent).opts(color='red')
  chart = chart * hv.HLine(mean_percent_75).opts(color='blue')
  chart = chart * hv.HLine(mean_percent_25).opts(color='blue')
  
  chart.opts(default_tools=["pan"])

  df_jumps_map = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'jumps_map'
    ).agg(pl.count('jumps_map').alias('count')) \
    .sort("count", descending=True)
  
  df_unijump = dfRows \
    .filter(
      pl.col(col) == test_value,
      pl.col(post_col).is_not_null()
    ) \
    .group_by(
      'unijump'
    ).agg(pl.count('unijump').alias('count')) \
    .sort("count", descending=True)


  print(f'df_viz FOR {col} = {test_value}')
  display(df_viz)

  display(chart)
  
  print(f'df_10_last FOR {col} = {test_value}')
  display(df_10_last)
  
  print(f'df_jumps_map FOR {col} = {test_value}')
  display(df_jumps_map)
  # display(df_unijump)

  print(f'df_unijump FOR {col} = {test_value}')
  display(df_unijump)

  lst_post_str_b = df_viz.filter(pl.col('percent') >= mean_percent)[post_col].to_list()
  lst_jumps_map = df_jumps_map['jumps_map'].to_list() # Used to discard
  
  return lst_post_str_b, lst_jumps_map

In [7]:
possible_b1, exclude_jump_maps_b1 = plot_post_col(col='str_b1', post_col='post_str_b1')

Setting test_value='10'
df_viz FOR str_b1 = 10


post_str_b1,count,percent
str,u32,f64
"""01""",54,17.821782
"""02""",33,10.891089
"""04""",31,10.231023
"""03""",31,10.231023
"""05""",29,9.570957
…,…,…
"""16""",2,0.660066
"""17""",2,0.660066
"""19""",1,0.330033


:Overlay
   .Bars.I    :Bars   [post_str_b1]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b1 = 10


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""10/04/2025""","""4182""","""30 33 34 10 21""","""1021303334""","[10, 21, … 34]",10,21,30,33,34,"""10""","""21""","""30""","""33""","""34""","""01""","""08""","""09""","""10""","""12""",false,0,"""11_09_03_01""",8
"""22/03/2025""","""4163""","""19 32 17 10 22""","""1017192232""","[10, 17, … 32]",10,17,19,22,32,"""10""","""17""","""19""","""22""","""32""","""04""","""08""","""10""","""13""","""15""",false,0,"""07_02_03_10""",0
"""16/03/2025""","""4157""","""26 10 21 16 30""","""1016212630""","[10, 16, … 30]",10,16,21,26,30,"""10""","""16""","""21""","""26""","""30""","""04""","""08""","""20""","""24""","""30""",true,8,"""06_05_05_04""",-2
"""21/02/2025""","""4134""","""32 28 22 10 30""","""1022283032""","[10, 22, … 32]",10,22,28,30,32,"""10""","""22""","""28""","""30""","""32""","""03""","""15""","""24""","""33""","""34""",true,12,"""12_06_02_02""",2
"""30/12/2024""","""4081""","""12 17 31 10 32""","""1012173132""","[10, 12, … 32]",10,12,17,31,32,"""10""","""12""","""17""","""31""","""32""","""01""","""13""","""15""","""16""","""24""",false,0,"""02_05_14_01""",-28
"""26/12/2024""","""4077""","""25 10 30 32 28""","""1025283032""","[10, 25, … 32]",10,25,28,30,32,"""10""","""25""","""28""","""30""","""32""","""12""","""18""","""20""","""23""","""28""",true,8,"""15_03_02_02""",-10
"""09/12/2024""","""4060""","""13 19 10 15 12""","""1012131519""","[10, 12, … 19]",10,12,13,15,19,"""10""","""12""","""13""","""15""","""19""","""05""","""08""","""15""","""20""","""29""",false,0,"""02_01_02_04""",-1
"""08/12/2024""","""4059""","""25 20 14 10 26""","""1014202526""","[10, 14, … 26]",10,14,20,25,26,"""10""","""14""","""20""","""25""","""26""","""10""","""12""","""13""","""15""","""19""",true,16,"""04_06_05_01""",0
"""01/12/2024""","""4052""","""10 31 20 30 26""","""1020263031""","[10, 20, … 31]",10,20,26,30,31,"""10""","""20""","""26""","""30""","""31""","""03""","""05""","""16""","""26""","""31""",false,0,"""10_06_04_01""",-3


df_jumps_map FOR str_b1 = 10


jumps_map,count
str,u32
"""01_02_04_14""",2
"""03_07_03_12""",2
"""01_04_13_07""",2
"""02_01_02_04""",2
"""09_03_02_03""",2
…,…
"""01_12_06_02""",1
"""07_07_01_02""",1
"""02_05_06_10""",1


df_unijump FOR str_b1 = 10


unijump,count
i64,u32
-3,12
-13,12
13,11
-1,11
-7,11
…,…
-39,1
-47,1
52,1


In [8]:
possible_b2, exclude_jump_maps_b2 = plot_post_col(col='str_b2', post_col='post_str_b2')

Setting test_value='15'
df_viz FOR str_b2 = 15


post_str_b2,count,percent
str,u32,f64
"""09""",32,8.39895
"""07""",29,7.611549
"""12""",26,6.824147
"""14""",26,6.824147
"""13""",23,6.036745
…,…,…
"""23""",5,1.312336
"""24""",2,0.524934
"""26""",1,0.262467


:Overlay
   .Bars.I    :Bars   [post_str_b2]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b2 = 15


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""05/04/2025""","""4177""","""34 33 15 05 16""","""0515163334""","[5, 15, … 34]",5,15,16,33,34,"""05""","""15""","""16""","""33""","""34""","""07""","""14""","""24""","""34""","""35""",true,16,"""10_01_17_01""",-57
"""10/03/2025""","""4151""","""29 06 26 15 19""","""0615192629""","[6, 15, … 29]",6,15,19,26,29,"""06""","""15""","""19""","""26""","""29""","""02""","""11""","""12""","""16""","""32""",true,16,"""09_04_07_03""",-15
"""22/02/2025""","""4135""","""34 33 15 24 03""","""0315243334""","[3, 15, … 34]",3,15,24,33,34,"""03""","""15""","""24""","""33""","""34""","""11""","""20""","""21""","""24""","""32""",true,8,"""12_09_09_01""",-11
"""20/02/2025""","""4133""","""34 15 26 04 30""","""0415263034""","[4, 15, … 34]",4,15,26,30,34,"""04""","""15""","""26""","""30""","""34""","""10""","""22""","""28""","""30""","""32""",true,8,"""11_11_04_04""",14
"""03/12/2024""","""4054""","""19 31 06 25 15""","""0615192531""","[6, 15, … 31]",6,15,19,25,31,"""06""","""15""","""19""","""25""","""31""","""05""","""06""","""11""","""16""","""23""",true,8,"""09_04_06_06""",-9
"""14/11/2024""","""4035""","""34 33 11 24 15""","""1115243334""","[11, 15, … 34]",11,15,24,33,34,"""11""","""15""","""24""","""33""","""34""","""12""","""14""","""16""","""21""","""23""",true,8,"""04_09_09_01""",-3
"""01/10/2024""","""3991""","""35 15 17 16 06""","""0615161735""","[6, 15, … 35]",6,15,16,17,35,"""06""","""15""","""16""","""17""","""35""","""01""","""02""","""22""","""28""","""31""",true,24,"""09_01_01_18""",9
"""06/09/2024""","""3966""","""35 18 02 34 15""","""0215183435""","[2, 15, … 35]",2,15,18,34,35,"""02""","""15""","""18""","""34""","""35""","""01""","""08""","""18""","""25""","""28""",false,0,"""13_03_16_01""",-51
"""03/09/2024""","""3963""","""15 33 14 20 29""","""1415202933""","[14, 15, … 33]",14,15,20,29,33,"""14""","""15""","""20""","""29""","""33""","""04""","""10""","""21""","""26""","""29""",false,0,"""01_05_09_04""",-9


df_jumps_map FOR str_b2 = 15


jumps_map,count
str,u32
"""01_05_09_04""",2
"""03_10_02_03""",2
"""11_12_01_04""",2
"""13_02_12_05""",2
"""08_01_01_18""",2
…,…
"""08_03_02_08""",1
"""14_06_06_03""",1
"""02_02_16_01""",1


df_unijump FOR str_b2 = 15


unijump,count
i64,u32
4,12
6,12
7,11
-1,11
3,10
…,…
35,1
-31,1
37,1


In [9]:
possible_b3, exclude_jump_maps_b3 = plot_post_col(col='str_b3', post_col='post_str_b3')

Setting test_value='23'
df_viz FOR str_b3 = 23


post_str_b3,count,percent
str,u32,f64
"""19""",28,7.368421
"""17""",25,6.578947
"""20""",23,6.052632
"""23""",22,5.789474
"""13""",21,5.526316
…,…,…
"""07""",5,1.315789
"""31""",5,1.315789
"""05""",4,1.052632


:Overlay
   .Bars.I    :Bars   [post_str_b3]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b3 = 23


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""07/02/2025""","""4120""","""08 35 23 28 17""","""0817232835""","[8, 17, … 35]",8,17,23,28,35,"""08""","""17""","""23""","""28""","""35""","""01""","""02""","""16""","""19""","""31""",false,0,"""09_06_05_07""",1
"""04/02/2025""","""4117""","""34 35 06 23 13""","""0613233435""","[6, 13, … 35]",6,13,23,34,35,"""06""","""13""","""23""","""34""","""35""","""14""","""19""","""28""","""31""","""34""",false,0,"""07_10_11_01""",-9
"""30/01/2025""","""4112""","""23 06 22 33 34""","""0622233334""","[6, 22, … 34]",6,22,23,33,34,"""06""","""22""","""23""","""33""","""34""","""04""","""05""","""19""","""28""","""32""",true,16,"""16_01_10_01""",-42
"""15/12/2024""","""4066""","""24 31 08 23 19""","""0819232431""","[8, 19, … 31]",8,19,23,24,31,"""08""","""19""","""23""","""24""","""31""","""05""","""07""","""11""","""14""","""29""",false,0,"""11_04_01_07""",5
"""09/11/2024""","""4030""","""26 23 30 12 17""","""1217232630""","[12, 17, … 30]",12,17,23,26,30,"""12""","""17""","""23""","""26""","""30""","""02""","""08""","""09""","""27""","""29""",false,0,"""05_06_03_04""",8
"""02/11/2024""","""4023""","""08 13 23 34 29""","""0813232934""","[8, 13, … 34]",8,13,23,29,34,"""08""","""13""","""23""","""29""","""34""","""09""","""13""","""14""","""22""","""34""",false,0,"""05_10_06_05""",12
"""31/10/2024""","""4021""","""24 12 03 23 35""","""0312232435""","[3, 12, … 35]",3,12,23,24,35,"""03""","""12""","""23""","""24""","""35""","""03""","""07""","""10""","""24""","""28""",false,0,"""09_11_01_11""",32
"""21/10/2024""","""4011""","""24 19 34 23 04""","""0419232434""","[4, 19, … 34]",4,19,23,24,34,"""04""","""19""","""23""","""24""","""34""","""04""","""11""","""24""","""28""","""33""",false,0,"""15_04_01_10""",4
"""08/10/2024""","""3998""","""12 02 32 23 28""","""0212232832""","[2, 12, … 32]",2,12,23,28,32,"""02""","""12""","""23""","""28""","""32""","""04""","""23""","""26""","""28""","""30""",false,0,"""10_11_05_04""",12


df_jumps_map FOR str_b3 = 23


jumps_map,count
str,u32
"""03_11_05_04""",2
"""05_01_05_01""",2
"""08_01_04_02""",2
"""03_16_01_02""",1
"""04_15_08_04""",1
…,…
"""12_01_11_01""",1
"""06_08_02_05""",1
"""03_18_02_06""",1


df_unijump FOR str_b3 = 23


unijump,count
i64,u32
2,13
7,13
19,11
-1,9
0,9
…,…
26,1
-23,1
54,1


In [10]:
possible_b4, exclude_jump_maps_b4 = plot_post_col(col='str_b4', post_col='post_str_b4')

Setting test_value='29'
df_viz FOR str_b4 = 29


post_str_b4,count,percent
str,u32,f64
"""26""",40,8.67679
"""24""",36,7.809111
"""25""",32,6.941432
"""28""",32,6.941432
"""27""",30,6.507592
…,…,…
"""13""",3,0.650759
"""10""",3,0.650759
"""09""",2,0.433839


:Overlay
   .Bars.I    :Bars   [post_str_b4]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b4 = 29


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""30/03/2025""","""4171""","""35 05 15 29 03""","""0305152935""","[3, 5, … 35]",3,5,15,29,35,"""03""","""05""","""15""","""29""","""35""","""07""","""16""","""22""","""26""","""34""",true,2,"""02_10_14_06""",-8
"""26/03/2025""","""4167""","""34 29 06 24 26""","""0624262934""","[6, 24, … 34]",6,24,26,29,34,"""06""","""24""","""26""","""29""","""34""","""05""","""08""","""27""","""32""","""35""",true,16,"""18_02_03_05""",-16
"""25/03/2025""","""4166""","""29 06 35 22 04""","""0406222935""","[4, 6, … 35]",4,6,22,29,35,"""04""","""06""","""22""","""29""","""35""","""06""","""24""","""26""","""29""","""34""",true,16,"""02_16_07_06""",31
"""24/03/2025""","""4165""","""32 29 27 21 06""","""0621272932""","[6, 21, … 32]",6,21,27,29,32,"""06""","""21""","""27""","""29""","""32""","""04""","""06""","""22""","""29""","""35""",false,0,"""15_06_02_03""",0
"""03/03/2025""","""4144""","""07 18 04 29 31""","""0407182931""","[4, 7, … 31]",4,7,18,29,31,"""04""","""07""","""18""","""29""","""31""","""09""","""14""","""16""","""22""","""24""",true,8,"""03_11_11_02""",-1
"""09/02/2025""","""4122""","""05 21 20 35 29""","""0520212935""","[5, 20, … 35]",5,20,21,29,35,"""05""","""20""","""21""","""29""","""35""","""07""","""19""","""21""","""25""","""32""",false,0,"""15_01_08_06""",-30
"""25/01/2025""","""4107""","""06 07 25 29 31""","""0607252931""","[6, 7, … 31]",6,7,25,29,31,"""06""","""07""","""25""","""29""","""31""","""03""","""09""","""30""","""32""","""35""",false,0,"""01_18_04_02""",43
"""17/01/2025""","""4099""","""02 14 27 32 29""","""0214272932""","[2, 14, … 32]",2,14,27,29,32,"""02""","""14""","""27""","""29""","""32""","""01""","""14""","""16""","""32""","""35""",false,0,"""12_13_02_03""",24
"""04/01/2025""","""4086""","""06 10 29 32 19""","""0610192932""","[6, 10, … 32]",6,10,19,29,32,"""06""","""10""","""19""","""29""","""32""","""07""","""19""","""20""","""23""","""28""",false,0,"""04_09_10_03""",-4


df_jumps_map FOR str_b4 = 29


jumps_map,count
str,u32
"""02_16_02_06""",2
"""04_19_04_06""",2
"""13_02_12_05""",2
"""18_08_01_02""",2
"""01_05_09_04""",2
…,…
"""04_05_15_01""",1
"""02_01_17_02""",1
"""02_18_01_03""",1


df_unijump FOR str_b4 = 29


unijump,count
i64,u32
-15,12
-4,11
-1,11
4,10
16,10
…,…
67,1
70,1
61,1


In [11]:
possible_blast, exclude_jump_maps_blast = plot_post_col(col='str_b5', post_col='post_str_blast')

Setting test_value='33'
df_viz FOR str_b5 = 33


post_str_blast,count,percent
str,u32,f64
"""35""",121,14.037123
"""33""",98,11.36891
"""34""",96,11.136891
"""32""",83,9.62877
"""31""",77,8.932715
…,…,…
"""16""",6,0.696056
"""15""",5,0.580046
"""14""",3,0.348028


:Overlay
   .Bars.I    :Bars   [post_str_blast]   (percent)
   .HLine.I   :HLine   [x,y]
   .HLine.II  :HLine   [x,y]
   .HLine.III :HLine   [x,y]

df_10_last FOR str_b5 = 33


fecha,sorteo,bolillas,id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_b2,post_str_b3,post_str_b4,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,str,str,str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,str,str,str,bool,u8,str,i64
"""01/04/2025""","""4173""","""12 25 33 13 08""","""0812132533""","[8, 12, … 33]",8,12,13,25,33,"""08""","""12""","""13""","""25""","""33""","""11""","""14""","""17""","""24""","""26""",false,0,"""04_01_12_08""",-29
"""21/03/2025""","""4162""","""32 33 01 17 26""","""0117263233""","[1, 17, … 33]",1,17,26,32,33,"""01""","""17""","""26""","""32""","""33""","""10""","""17""","""19""","""22""","""32""",false,0,"""16_09_06_01""",-6
"""28/02/2025""","""4141""","""33 04 28 22 07""","""0407222833""","[4, 7, … 33]",4,7,22,28,33,"""04""","""07""","""22""","""28""","""33""","""02""","""05""","""11""","""25""","""32""",true,16,"""03_15_06_05""",29
"""17/02/2025""","""4130""","""12 33 22 19 20""","""1219202233""","[12, 19, … 33]",12,19,20,22,33,"""12""","""19""","""20""","""22""","""33""","""01""","""05""","""13""","""20""","""26""",false,0,"""07_01_02_11""",1
"""16/02/2025""","""4129""","""33 07 12 04 32""","""0407123233""","[4, 7, … 33]",4,7,12,32,33,"""04""","""07""","""12""","""32""","""33""","""12""","""19""","""20""","""22""","""33""",false,0,"""03_05_20_01""",-47
"""13/02/2025""","""4126""","""17 30 21 33 15""","""1517213033""","[15, 17, … 33]",15,17,21,30,33,"""15""","""17""","""21""","""30""","""33""","""09""","""14""","""15""","""26""","""28""",false,0,"""02_04_09_03""",-14
"""12/02/2025""","""4125""","""31 33 20 12 13""","""1213203133""","[12, 13, … 33]",12,13,20,31,33,"""12""","""13""","""20""","""31""","""33""","""15""","""17""","""21""","""30""","""33""",false,0,"""01_07_11_02""",-11
"""03/02/2025""","""4116""","""17 31 32 33 19""","""1719313233""","[17, 19, … 33]",17,19,31,32,33,"""17""","""19""","""31""","""32""","""33""","""06""","""13""","""23""","""34""","""35""",true,8,"""02_12_01_01""",32
"""28/12/2024""","""4079""","""28 33 03 22 32""","""0322283233""","[3, 22, … 33]",3,22,28,32,33,"""03""","""22""","""28""","""32""","""33""","""01""","""07""","""15""","""30""","""34""",true,16,"""19_06_04_01""",-12


df_jumps_map FOR str_b5 = 33


jumps_map,count
str,u32
"""13_06_02_06""",2
"""12_10_04_01""",2
"""23_04_02_01""",2
"""08_19_01_03""",2
"""04_09_12_01""",2
…,…
"""03_05_04_18""",1
"""01_06_12_11""",1
"""04_12_05_06""",1


df_unijump FOR str_b5 = 33


unijump,count
i64,u32
0,22
-1,20
5,20
2,20
10,20
…,…
-43,1
-58,1
-59,1


In [12]:
b_count = rr.CHOOSE
min_b = rr.MIN_B_VALUES[0]
max_b = rr.MAX_B_VALUES[-1]


combos = combinations(range(min_b, max_b + 1), b_count)
# print("generating combos: ", len(list(combos)))

In [13]:
lst = []
post_str_b1 = None
post_str_blast = None
for row in combos:
  cols = []
  str_row = " ".join([str(r).zfill(2) for r in row])
  id, bolillas_int, str_combo = formalize_data_with_str(bolillas_str=str_row)
  cols.append(id)
  cols.append(bolillas_int)
  cols.extend([b for b in bolillas_int])
  cols.extend(str_combo)
  cols.append(post_str_b1)
  cols.append(post_str_blast)

  is_rare, reason = rr.is_rare_combination(bolillas_int, id)
  cols.append(is_rare)
  cols.append(reason)

  cols.append(rr.jumps_map(bolillas_int))
  cols.append(rr.unijump(bolillas_int))
  
  lst.append(cols)

  post_str_b1 = str_combo[0]
  post_str_blast = str_combo[-1]

with open('./gn_cahe_v3.json', 'w') as file: 
  json.dump(lst, file)

df_Orows = pl.DataFrame(
  lst,
  schema=[
    # 'fecha', 'sorteo', 'bolillas',
    'id', 'combo', 'b1', 'b2', 'b3', 'b4', 'b5',
    'str_b1', 'str_b2', 'str_b3', 'str_b4', 'str_b5',
    'post_str_b1', 'post_str_blast',
    'is_rare_combination', 'reason',
    "jumps_map", "unijump"
  ],
  schema_overrides={
    'combo': pl.Array(pl.UInt8, 5),
    'b1': pl.UInt8,
    'b2': pl.UInt8,
    'b3': pl.UInt8,
    'b4': pl.UInt8,
    'b5': pl.UInt8,
    'b6': pl.UInt8,
    'reason': pl.UInt8
  },
  orient='row'
)

TOTAL_LINES = len(df_Orows)
print(f'{TOTAL_LINES=}')
display(df_Orows.head(10))


TOTAL_LINES=324632


id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0102030405""","[1, 2, … 5]",1,2,3,4,5,"""01""","""02""","""03""","""04""","""05""",null,null,true,8,"""01_01_01_01""",0
"""0102030406""","[1, 2, … 6]",1,2,3,4,6,"""01""","""02""","""03""","""04""","""06""","""01""","""05""",true,8,"""01_01_01_02""",1
"""0102030407""","[1, 2, … 7]",1,2,3,4,7,"""01""","""02""","""03""","""04""","""07""","""01""","""06""",true,8,"""01_01_01_03""",2
"""0102030408""","[1, 2, … 8]",1,2,3,4,8,"""01""","""02""","""03""","""04""","""08""","""01""","""07""",true,8,"""01_01_01_04""",3
"""0102030409""","[1, 2, … 9]",1,2,3,4,9,"""01""","""02""","""03""","""04""","""09""","""01""","""08""",true,8,"""01_01_01_05""",4
"""0102030410""","[1, 2, … 10]",1,2,3,4,10,"""01""","""02""","""03""","""04""","""10""","""01""","""09""",true,8,"""01_01_01_06""",5
"""0102030411""","[1, 2, … 11]",1,2,3,4,11,"""01""","""02""","""03""","""04""","""11""","""01""","""10""",true,8,"""01_01_01_07""",6
"""0102030412""","[1, 2, … 12]",1,2,3,4,12,"""01""","""02""","""03""","""04""","""12""","""01""","""11""",true,8,"""01_01_01_08""",7
"""0102030413""","[1, 2, … 13]",1,2,3,4,13,"""01""","""02""","""03""","""04""","""13""","""01""","""12""",true,8,"""01_01_01_09""",8


In [14]:
# possible_b1, exclude_jump_maps_b1

df_possible = df_Orows.filter(
  pl.col("str_b1").is_in(possible_b1),
  pl.col("str_b2").is_in(possible_b2),
  pl.col("str_b3").is_in(possible_b3),
  pl.col("str_b4").is_in(possible_b4),
  # ~pl.col("str_b1").is_in(['18', '19', '20', '21', '22', '23', '24', '26']),
  ~pl.col("jumps_map").is_in(exclude_jump_maps_b1 + exclude_jump_maps_b2 + exclude_jump_maps_b3 + exclude_jump_maps_b4 + exclude_jump_maps_blast),
  pl.col("str_b5").is_in(possible_blast),
  pl.col("is_rare_combination") == False
)

display(df_possible)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0105111932""","[1, 5, … 32]",1,5,11,19,32,"""01""","""05""","""11""","""19""","""32""","""01""","""31""",false,0,"""04_06_08_13""",3
"""0105112030""","[1, 5, … 30]",1,5,11,20,30,"""01""","""05""","""11""","""20""","""30""","""01""","""29""",false,0,"""04_06_09_10""",-3
"""0105112031""","[1, 5, … 31]",1,5,11,20,31,"""01""","""05""","""11""","""20""","""31""","""01""","""30""",false,0,"""04_06_09_11""",-2
"""0105112032""","[1, 5, … 32]",1,5,11,20,32,"""01""","""05""","""11""","""20""","""32""","""01""","""31""",false,0,"""04_06_09_12""",-1
"""0105112033""","[1, 5, … 33]",1,5,11,20,33,"""01""","""05""","""11""","""20""","""33""","""01""","""32""",false,0,"""04_06_09_13""",0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""0818243132""","[8, 18, … 32]",8,18,24,31,32,"""08""","""18""","""24""","""31""","""32""","""08""","""35""",false,0,"""10_06_07_01""",-12
"""0818243133""","[8, 18, … 33]",8,18,24,31,33,"""08""","""18""","""24""","""31""","""33""","""08""","""32""",false,0,"""10_06_07_02""",-11
"""0818243134""","[8, 18, … 34]",8,18,24,31,34,"""08""","""18""","""24""","""31""","""34""","""08""","""33""",false,0,"""10_06_07_03""",-10


In [15]:
df_possible.sample(n=1)

id,combo,b1,b2,b3,b4,b5,str_b1,str_b2,str_b3,str_b4,str_b5,post_str_b1,post_str_blast,is_rare_combination,reason,jumps_map,unijump
str,"array[u8, 5]",u8,u8,u8,u8,u8,str,str,str,str,str,str,str,bool,u8,str,i64
"""0218202433""","[2, 18, … 33]",2,18,20,24,33,"""02""","""18""","""20""","""24""","""33""","""02""","""32""",false,0,"""16_02_04_09""",-13
